In [4]:
pip install opencv-python

   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 196.9 kB/s eta 0:03:16
   ---------------------------------------- 0.1/38.6 MB 273.8 kB/s eta 0:02:21
   ---------------------------------------- 0.1/38.6 MB 327.7 kB/s eta 0:01:58
   ---------------------------------------- 0.1/38.6 MB 344.8 kB/s eta 0:01:52
   ---------------------------------------- 0.1/38.6 MB 343.4 kB/s eta 0:01:53
   ---------------------------------------- 0.2/38.6 MB 419.0 kB/s eta 0:01:32
   ---------------------------------------- 0.2/38.6 MB 436.8 kB/s eta 0:01:28
   ---------------------------------------- 0.2/38.6 MB 444.3 kB/s eta 0:01:27
   ---------------------------------------- 0.2/38.6 MB 443.5 kB/s eta 0:01:27
   ------

In [2]:
pip install pygame


   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 435.7 kB/s eta 0:00:25
   ---------------------------------------- 0.0/10.8 MB 281.8 kB/s eta 0:00:39
   ---------------------------------------- 0.1/10.8 MB 328.2 kB/s eta 0:00:33
   ---------------------------------------- 0.1/10.8 MB 328.2 kB/s eta 0:00:33
   ---------------------------------------- 0.1/10.8 MB 245.8 kB/s eta 0:00:44
   ---------------------------------------- 0.1/10.8 MB 245.8 kB/s eta 0:00:44
   ---------------------------------------- 0.1/10.8 MB 228.2 kB/s eta 0:00:47
   ---------------------------------------- 0.1/10.8 MB 228.2 kB/s eta 0:00:47
   ---------------------------------------- 0.1/10.8 MB 233.8 kB/s eta 0:00:46
   ---------------------------------------- 0.1/10.8 MB 233.8 kB/s eta 0:00:46


In [7]:
import cv2
from pygame import mixer
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np


In [8]:
#use haarcascade classifier to detect faces and eyes
#it will return [x,y,w,h] coordinates which we will border to detect eyes and face on camera
#then frame() i.e download each eye frame into variable the resize ,reshape , rescale the image of eye
#model prediction 
# model.predict() on frame i.e downloaded
#  if we print prediction it will give us ndim=2 tensor , with two values . If value >0.5 eyes open , <0.5 eyes closed
# so we make a black box if value of prediction is >0.5 it shloud be printed close if it is less than 0.5 then close
# maintain a score ==0 if eyes are closed score++ ;  else score--
# if score >20
# play alarm

In [9]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')
model = load_model(r'D:\ML_projects\project_data\Model\model.h5')

In [10]:
mixer.init()
sound = mixer.Sound(r'D:\ML_projects\project_data\Code\alarm.wav')
cap = cv2.VideoCapture(0)
score = 0
while True:
    ret,frame = cap.read()
    # frame = cv2.flip(frame,1)
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=2)

    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    # first detect face then eye on that face
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color = (255,0,0),thickness=2)
        # color: blue,left-bottommost point to right topmost ; frame means coloured format(BGR)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh),color = (255,0,0),thickness=2)

        # preprocessing steps
        eye = frame[y:y+h,x:x+w]
        eye = cv2.resize(eye,(80,80))
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye,axis=0)

        # model prediction
        prediction = model.predict(eye)
        print(prediction)

        # close eyes
        if prediction[0][0]>0.50:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255),
                       thickness=1)
            cv2.putText(frame,'Score:'+ str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255),
                       thickness=1)
            score =score+1
            if score>20:
                try:
                    sound.play()
                except:
                    pass
        # open eyes
        elif prediction[0][1]>0.50:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255),
                       thickness=1)
            cv2.putText(frame,'Score:'+str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=1,color=(255,255,255),
                       thickness=1)
            score = score-1
            if score<0:
                score = 0
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 2s 2s/step
[[9.9925870e-01 7.4126886e-04]]
1/1 [==============================] - 0s 75ms/step
[[9.9999583e-01 4.1186713e-06]]
1/1 [==============================] - 0s 73ms/step
[[0.9808307 0.0191692]]
1/1 [==============================] - 0s 82ms/step
[[9.9989021e-01 1.0977413e-04]]
1/1 [==============================] - 0s 61ms/step
[[0.01031642 0.9896835 ]]
1/1 [==============================] - 0s 68ms/step
[[0.99751115 0.0024888 ]]
1/1 [==============================] - 0s 68ms/step
[[0.00137487 0.99862516]]
1/1 [==============================] - 0s 66ms/step
[[0.67656714 0.32343283]]
1/1 [==============================] - 0s 66ms/step
[[0.17995907 0.82004094]]
1/1 [==============================] - 0s 68ms/step
[[0.00448951 0.99551046]]
1/1 [==============================] - 0s 65ms/step
[[1.6741151e-09 1.0000000e+00]]
1/1 [==============================] - 0s 68ms/step
[[0.14687696 0.85312307]]
1/1 [==============================] - 0s 82